# Week 3

In this notebook we are going to implement the [finite difference time domain method](https://en.wikipedia.org/wiki/Finite-difference_time-domain_method) to advance the electromagnetic field in vacuum by solving Maxwell-Faraday and Maxwell-Ampere equations. For simplicity, we consider a one-dimensional domain (e.g. field vary only along z). We will model an electromagnetic pulse travelling along z. This means that the pulse will be characterised by Ex and By.

First we import all the modules that we will need.

In [ ]:
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import copy

Let's define a class called `EMfield`. The class will contain:
<br>
- the `__init__` method to set the initial state of the object
<br>
- an `advanceE` method, which solve the discretised Faraday equation
<br>
- an `advanceB` method, which solve the discretised Ampere equation
<br>
- an `periodic_bs` method, which appies periodic boundary conditions to the electric field

In [ ]:
class EMfield:
    
    def __init__(self, Ex, By):
        self.Ex = Ex
        self.By = By
        
    def advanceE(self, cfl, N):

        """
        advanceE
        
        Parameters:
        ----------
        cfl: float
            cfl condition
        N: integer
            number of grid points
        """
        
        for i in range(1,N-1):
            self.Ex[i] = self.Ex[i] + cfl * (self.By[i-1] - self.By[i])
            
    def advanceB(self, cfl, N):
        
        """
        advanceB
        
        Parameters:
        ----------
        cfl: float
            cfl condition
        N: integer
            number of grid points
        """
        
        for i in range(0,N-1):
            self.By[i] = self.By[i] + cfl * (self.Ex[i] - self.Ex[i+1])
    
    def periodic_bc(self, cfl, N):
        
        """
        periodic_bc
        
        Parameters:
        ----------
        cfl: float
            cfl condition
        N: integer
            number of grid points
        """
            
        self.Ex[0] = self.Ex[0] + cfl * (self.By[N-2] - self.By[0])
        self.Ex[N-1] = self.Ex[N-1] + cfl * (self.By[N-2] - self.By[0])

Now we write the main programme. Here we define the qauntities that we need (grid, grid size, time step...). We initailise Ex and By. According to the method, Ex is defined on the nodes of the grid, while By at the centre of the cell. The main programme will then contain the temporal loop to advance the fields.
<br>
We create a list called `data` where we store our results. In particular we store the time, the list containing the grid points, the list containing cell centre points and the object `EMfield`. We store data every 10 time iterations to reduce the size of this list. Finally we save `data` into a `.npy` file.

In [ ]:
L = 1.0 # domain size of the simulation in normalised units
c = 1.0 # speed of light in normalised units
dz = 0.0025 # space discretisation
N = int(math.ceil(L/dz)) + 1 # number of grid points
z = [i*dz for i in range(0,N)]
zc = [z[i]+dz/2 for i in range(0,N-1)]
Nhalf = int(math.ceil(N/2)) # location for the electromagentic puls
cfl = 0.99 # c*dt/dx
dt = cfl*dz/c # time step
Nt = 2000 # number of temporal steps
t = 0 # initial tiime

Ex = [0 for _ in range(0,N)] # electric field initialisation
By = [0 for _ in range(0,N-1)] # magnetic field initialisation
field = EMfield(Ex,By) # class initialisation

Data = []

# temporal loop to advance fields
for i in range(0,Nt):
    
    print("Iteration {} out of {}".format(i,Nt))
    
    t = t + dt
    
    field.advanceE(cfl,N) # solve E
    
    field.periodic_bc(cfl,N) # apply boundary conditions
    
    pulse = math.exp(-0.5 * ( (40*dt - t)/(12*dt) )**2) 
    field.Ex[Nhalf] = field.Ex[Nhalf] + pulse # add the electromagentic pulse
    
    field.advanceB(cfl,N) # solve B
    
    # store information in a list
    if (i%10 == 0):
        my_list = [t, z, zc, copy.deepcopy(field)]
        Data.append(my_list)

np.save("field", Data, allow_pickle=True) # save list to a file called field

Now let's write a small piece of code to load the data saved in the file `field.npy` and plot the content. We want to see the evolution in time of Ex(z) and By(z). So we use a special method of `matplotlib`.

In [ ]:
data = np.load("field.npy", allow_pickle = True) # load the file and store in a variable called data

# instructions to set the figure
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,1,1)
ax1.set_xlim(0.0,1.0)
ax1.set_ylim(-1,1)
ax1.set_ylabel("$E_x$",fontsize=20)
ax1.tick_params(labelsize=20)
plot1, = ax1.plot([],[],'k')
ax2 = fig.add_subplot(2,1,2)
ax2.set_xlim(0.0,1.0)
ax2.set_ylim(-1,1)
ax2.set_ylabel("$B_y$",fontsize=20)
ax2.set_xlabel("$z$",fontsize=20)
ax2.tick_params(labelsize=20)
plot2, = ax2.plot([],[],'r')

# function to set the plot
def anim_step(i):

    # unpacking information from data
    t = data[i][0]
    z = data[i][1]
    zc = data[i][2]
    ex = data[i][3].Ex
    by = data[i][3].By
    
    plot1.set_data(z,ex)
    plot2.set_data(zc,by)
    
    return plot1, plot2,

# special instructions to show an animation
anim = FuncAnimation(fig, anim_step, frames = 200, interval = 200, blit = True, repeat=False)

from IPython.display import HTML
HTML(anim.to_jshtml())

Now add a new method in `EMfield` to apply [perfect conductor boundary condtions](https://doc.comsol.com/5.5/doc/com.comsol.help.rf/rf_ug_radio_frequency.07.11.html#:~:text=Perfect%20Electric%20Conductor&text=is%20a%20special%20case%20of,a%20symmetry%20type%20boundary%20condition.https://doc.comsol.com/5.5/doc/com.comsol.help.rf/rf_ug_radio_frequency.07.11.html#:~:text=Perfect%20Electric%20Conductor&text=is%20a%20special%20case%20of,a%20symmetry%20type%20boundary%20condition.). This means that Ex is set to zero at the boundary.